In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import pandas as pd
import numpy as np
import quandl
import cvxpy as cvx

In [3]:
sys.path.insert(0, os.path.abspath('..'))
import helium

In [4]:
%matplotlib inline

In [5]:
quandl.ApiConfig.api_key = '8wnB7EcvJm1xEvJjzWBR'

In [6]:
tickers = ['AMZN', 'GOOGL', 'TSLA', 'NKE']
start_date='2012-01-01'
end_date='2016-12-31'
returns = pd.DataFrame(dict([(ticker, quandl.get('WIKI/'+ticker, 
                                    start_date=start_date, 
                                    end_date=end_date)['Adj. Close'].pct_change())
                for ticker in tickers]))
returns[["_CASH"]]=quandl.get('FRED/DTB3', start_date=start_date, end_date=end_date)/(250*100)
returns = returns.fillna(method='ffill').iloc[1:]

returns.tail()

,AMZN,GOOGL,NKE,TSLA,_CASH
Date,,,,,
2016-12-23,-0.007503,-0.002322,-0.004411,0.023459,0.000020
2016-12-27,0.014213,0.002637,-0.011944,0.029015,0.000020
2016-12-28,0.000946,-0.006618,-0.005264,0.000957,0.000021
2016-12-29,-0.009040,-0.002101,0.000784,-0.023027,0.000018
2016-12-30,-0.019970,-0.012991,-0.004505,-0.004612,0.000020


In [7]:
rets = returns.rolling(window=250, min_periods=250).mean().shift(1).dropna()
sigmas = returns.rolling(window=250, min_periods=250).cov().unstack().shift(1).stack()

rets.tail()

,AMZN,GOOGL,NKE,TSLA,_CASH
Date,,,,,
2016-12-23,0.000683,0.000219,-0.000660,-0.000074,0.000012
2016-12-27,0.000542,0.000150,-0.000706,-0.000124,0.000013
2016-12-28,0.000627,0.000179,-0.000691,-0.000023,0.000013
2016-12-29,0.000707,0.000215,-0.000664,-0.000052,0.000013
2016-12-30,0.000901,0.000302,-0.000598,0.000133,0.000013


In [8]:
cash_ticker = "_CASH"

In [9]:
price_sigma = rets.copy()
price_sigma[:] = 0.

In [10]:
volumes = pd.DataFrame(dict([(ticker, quandl.get('WIKI/'+ticker, 
                                    start_date=start_date, 
                                    end_date=end_date)['Adj. Volume'])
                for ticker in tickers]))

In [11]:

volumes[cash_ticker] = np.finfo(np.float).max

In [12]:
tcost_model=helium.TransactionCost(gamma = 1., 
                                   half_spread=10E-4,
                                   nonlin_coef = 0.,  #1.,
                                   sigmas = price_sigma, 
                                   nonlin_power = 1.5,
                                   volumes = volumes,
                                   asym_coef = 0.)

In [13]:
borrow_costs = rets.copy()
borrow_costs[:] = 1E-4
borrow_costs["_CASH"] = 0.

dividends = rets.copy()
dividends[:] = 0.


hcost_model=helium.HoldingCost(gamma = 1., borrow_costs=borrow_costs, dividends=dividends)

In [14]:
risk_model = helium.BasicRiskCost(5.0, sigmas)

In [15]:
leverage_limit = helium.LeverageLimitConstraint(3.)

In [16]:
simulator = helium.MarketSimulator(rets, volumes, costs=[tcost_model, hcost_model], cash_ticker='_CASH')

In [17]:
holding_init = pd.Series(index=rets.columns, data=0.)
holding_init[cash_ticker] = 100.
holding_init

AMZN       0.0
GOOGL      0.0
NKE        0.0
TSLA       0.0
_CASH    100.0
dtype: float64

In [18]:
r = helium.DefaultRet(rets, None)

In [19]:
spo_policy = helium.SinglePeriodOpt(return_forecast=r,
                                   costs=[risk_model, tcost_model, hcost_model], 
                                   constraints=[leverage_limit]
                                   )

In [20]:
sigmas.loc[pd.Timestamp('2013-01-03 00:00:00')].values

array([[ 3.99561230e-04,  1.00539021e-04,  4.99997151e-05,
         1.40325073e-04, -6.53060286e-11],
       [ 1.00539021e-04,  2.06562688e-04,  2.62686927e-05,
         1.13564454e-04,  1.73824862e-09],
       [ 4.99997151e-05,  2.62686927e-05,  2.11768678e-04,
         1.17747593e-04, -1.00539542e-09],
       [ 1.40325073e-04,  1.13564454e-04,  1.17747593e-04,
         1.13136968e-03,  3.48403789e-10],
       [-6.53060286e-11,  1.73824862e-09, -1.00539542e-09,
         3.48403789e-10,  7.22827309e-13]])

In [38]:
?cvx.abs

In [37]:
result = simulator.run(holding_init, policy=spo_policy, start_date='2013-01-05', end_date='2016-12-31')

INFO:root:Backtest started, from 2013-01-07 00:00:00 to 2016-12-30 00:00:00
INFO:root:Getting trades at time <module 'time' (built-in)>
INFO:root:Propagating portfolio at time 2013-01-03 00:00:00


TypeError: <class 'pandas.core.series.Series'> is not a valid type for a Constant value.